# Introduction to the Prediktor Edge Client python library

This library allows programatic control to the Prediktor Apis Edge components through the Python programming language. It is possible to manipulate Hive istances, Modules, Items, Properties, Attributes, Honestore databases. The supported actions are add, access, remove/delete (where applicable) and write.



## Background

The PyPrediktorEdgeClient library allows the user to connect to an Apis instances. It is possible to create and manipulate Apis modules and items. The following core operations are available

* Apis Hive Service instances
  * Creating an Apis Hive service
  * Starting and stopping an Apis Hive service
  * Deleting an Apis Hive service
* Apis Hive operations
  * Creating modules
  * Listing Apis modules
  * Reading data as VQT records
* Apis Hive Module operations
  * Accessing module properties
  * Setting/getting module properties directly
  * Listing and retrieving module items
* Apis Hive Item operations
  * Accessing item attributes
  * Setting/getting module attributes directly


## Notes and warnings

---
**NOTE**

This library gives almost unrestricted access to the Apis Edge. Please note that operations on Apis Edge are final, and it is possible to delete or damage vital parts of a working system. Please use with care!

---

---
**NOTE**

This library is open source and not supported by Prediktor. Prediktor assumes no liability or responsibility regarding the functionality or correctness of the code.

---


## Installation

```
pip install git+https://github.com/PrediktorAS/PyPrediktorEdgeClient.git
```

See the readme.md as well as [The PythonNet Github page](https://github.com/pythonnet/pythonnet/wiki/Troubleshooting-on-Windows,-Linux,-and-OSX) for further installation instructions.

## Library structure

The library is structured as a package with several subpackages:

    pyprediktoredgeclient
        hive
        hiveservices
        util
        honeystore^*
        timeseries^*
        chronicle^*

*) Not completed yet

## API conventions

There are several conventions used in the design of the *Python interface for Apis Edge*, `pyprediktoredgeclient`. The API follows the conventions set out in
[PEP8](https://www.python.org/dev/peps/pep-0008/), but with relaxation regarding line length.

* Variable, function and method names uses `snake_case` -- lower case names joined with underscore
* Classes are CamelCase
* Method names are structured as *verb_subject*, i.e. `get_item('')`
* Attributes (as python language construct, not to be confused with item attributes) are constructed from subjects or adjectives (i.e. `hive.module_types`).

# Hive services

Hive services is concerned with the representation of Hive instances on the computer, not the implementation of logic on that Hive instance.

## Importing the modules

The hive instances are accessed through the `hiveservices` module. The different modules such as `hive`, `hiveservices` etc relies on the `util`
package. This package handles import of the various Apis Assemblies as a part of the module loading. The asseblies are loaded according to the
following sequence:

1. If the environment variable 'APIS_INSTALL_LOCATION' is defined - use that as the import location
2. If running on a windows platform check for possible, common installation locations of Apis in the registry and use those to 
   load the assemblies
3. Install the assemblies from the install kit

The location where the asseblies are loaded from are located in the variable `util.importlocation`


In [8]:
from pyprediktoredgeclient import hiveservices, util
util.importlocation, util.imported_assemblies

('C:\\Program Files\\APIS\\Bin64',
 ['C:\\Program Files\\APIS\\Bin64\\HiveNetApi.dll',
  'C:\\Program Files\\APIS\\Bin64\\ApisNetUtilities.dll',
  'C:\\Program Files\\APIS\\Bin64\\Microsoft.Win32.Registry.dll',
  'C:\\Program Files\\APIS\\Bin64\\HoneystoreNetApi.dll',
  'C:\\Program Files\\APIS\\Bin64\\Prediktor.Log.dll',
  'C:\\Program Files\\APIS\\Bin64\\SentinelRMSCore.dll'])

##  List all hive instances UUIDs on the machine

In [9]:
uuids = hiveservices.instance_identifiers()
uuids

[UUID('51f92300-ca68-11d2-85c3-0000e8404a66'),
 UUID('04d4eb61-ccd3-43cc-a4ca-d2b5c4c56b05'),
 UUID('1839a793-bc84-4825-a25b-2b6dd4f5b333'),
 UUID('4941b1b0-e948-4788-8fcb-1e7dc9ce6a67'),
 UUID('8322407c-ea3d-4886-9d99-cc8ccd35da6f'),
 UUID('8840f50b-924d-4429-bc52-d67840169db8'),
 UUID('dedc70ab-ad1b-4de2-9e56-4e53f280b342'),
 UUID('f84849c5-e05e-4666-a290-5792b0448a50'),
 UUID('fb08a90b-a355-47c7-b3b8-a3328e8dce4a')]

## List all instances 

The `hiveservices.list_instances()`function returns a list of `HiveInstance` objects. The main properties of a `HiveInstance` object is:

* `name`: The name of the instance
* `prog_id`: The prog id (Com class name) of the Hive instance
* `running`: A bool property indicating wheter the instance is running or not
* `CLSID`: The UUID class id.

In [10]:
instance_list = hiveservices.list_instances()

for inst in instance_list:
    print(f"{inst.name}\n\t{inst.running}\t{inst.CLSID}\t{inst.prog_id}\n")

ApisHive
	False	51f92300-ca68-11d2-85c3-0000e8404a66	Prediktor.ApisLoader.1

ScadaInput
	False	04d4eb61-ccd3-43cc-a4ca-d2b5c4c56b05	Prediktor.ApisLoader.ScadaInput

MaerskSource
	False	1839a793-bc84-4825-a25b-2b6dd4f5b333	Prediktor.ApisLoader.MaerskSource

MaerskLogger
	False	4941b1b0-e948-4788-8fcb-1e7dc9ce6a67	Prediktor.ApisLoader.MaerskLogger

Kontornett
	False	8322407c-ea3d-4886-9d99-cc8ccd35da6f	Prediktor.ApisLoader.Kontornett

AggregatedTags
	False	8840f50b-924d-4429-bc52-d67840169db8	Prediktor.ApisLoader.AggregatedTags

pytestinstance
	True	dedc70ab-ad1b-4de2-9e56-4e53f280b342	Prediktor.ApisLoader.pytestinstance

Simulator
	False	f84849c5-e05e-4666-a290-5792b0448a50	Prediktor.ApisLoader.Simulator

BachalauSim
	False	fb08a90b-a355-47c7-b3b8-a3328e8dce4a	Prediktor.ApisLoader.BachalauSim



check that the list of UUID's and instance_list are same length and that all UUIDs are available

In [11]:
assert len(uuids) == len(instance_list)

#check that all instances in instance list has a corresponding UUID
for instance in instance_list:
    assert instance.CLSID in uuids

## Creating a new instance

We'll try to fetch the instance 'pytest' and create it if it doesn't exist

In [12]:

try:
    pytestinstance = hiveservices.get_instance('pytestinstance')
except hiveservices.Error:
    pytestinstance = hiveservices.add_instance('pytestinstance')

# check that 'pytest' is among the instances

assert 'pytestinstance' in [instance.name for instance in hiveservices.list_instances()]

## Starting and stopping services

The running state of Hive instances can be interrogated through the `.running` attribute of an `ApisInstance` object

In [13]:
if pytestinstance.running:
    print("'pytestinstance' instance is running")
else:
    print("'pytestinstance' instance was stopped, starting")

pytestinstance.running = True    

'pytestinstance' instance is running


# Hive, Modules and Items

The `hive` subpackage gives the user access to the Apis Edge hive and related parts such as modules and items. The `hive` functionality must not be confused with `hiveservices` mentioned above. The classes and functions in `hive` is concerned with the configuration of the system whereas `hiveservice` deals with the management of the hive on the computer, such as stopping and starting the system.

Some clarifications and definitions might be in order. The main classes defined in the `hive`package are:

* `Hive` This class contains the configuration of one hive. A hive is an insulated service that manages input, output, logging and processing. A `Hive` contains any number of `Module`s.
* `Module` This class contains all the `Item`s associated with one functionality. A functionality could be the OPC connection to a particular server. The parameters that control the `Module` behaviour are called `Property`. 
* `Property` The parameters for one `Module` such as ExchangeRate.
* `Item` contains the real-time information in the system. The various parameters and values associated with one `Item` are known as `Attribute`s

## Importing the package and connecting to a Hive

In this example we use the `hiveinstance` object from the previous section to connect to the `hive` however we could also use the prog-id of the hive as a connection parameter, 

In [14]:
from pyprediktoredgeclient import hive
test_hive = hive.Hive(pytestinstance)           #alternative: test_hive = hive.Hive('Prediktor.ApisLoader.pytestinstance')
assert test_hive.name=='pytestinstance'

## The *module types*

A hive will support several *module types*. Each module type takes care of a given aspect of the Hive configuration, an *OPC-UA client module* will used to connect to a remote computer, a *Worker module* will be used to perform calculations etc. We can retrive the supported module types with the `.module_types` attribute of the hive object.

In [8]:
mod_class_names = [mod_type.class_name for mod_type in test_hive.module_types]
assert 'ApisWorker' in mod_class_names

Lets print out the first ten module class names

In [9]:
print('\n'.join(mod_class_names[:10]))
worker_type = test_hive.get_module_type('ApisWorker')

ApisOPC
ApisTaskScheduler
ApisHAGovernor
ApisBatchOptimizerBee
ApisIntegoBee
ApisMessageBuilderBee
ApisModbusSlave
ApisLVEstimator2
ApisGPSolarBee
ApisHSMirror


## Accessing, adding and removing modules from the hive.

The modules in the hive can be retrived directly by name or numeric index by subscripting, (i.e. `mod = hive['mymodule']`) or by iterating over the hive (i.e. `for mod in hive: ...`). Modules are added and removed from a hive using `.add_module(module_type, module_name)` and `del hive['module_name']` respectivly

In [10]:
## deleting all modules with "testworker" in the name
for mod in test_hive:
    if "testworker" in mod.name:
        mod.delete()            #we could have used del test_hive[mod] or del test_hive[mod.name] as well. 

module_names = [mod.name for mod in test_hive]

creating two modules. We either use a `ModuleType` instance or a string as the module_type parameter

In [11]:
test_module= test_hive.add_module(worker_type, 'testworker', exchangerate=2000)
test_module2 = test_hive.add_module('ApisWorker', 'testworker2')

test_modules = [mod for mod in test_hive if 'testworker' in mod.name]
assert len(test_modules)==2

## Setting module properties

The module properties can be accessed directly

In [12]:
assert test_module.exchangeRate == 2000

test_module.ExchangeRate=1000

assert test_module.get_property('eXchangeRate').value == 1000


In [13]:
item_type_names = [item_type.Name for item_type in test_module.item_types]

assert 'Signal' in item_type_names
assert 'Variable' in item_type_names
assert 'Function item' in item_type_names
item_type_names

['Signal',
 'Time',
 'Variable',
 'VariableVector',
 'BitSelect',
 'String formatter',
 'Multiplexer',
 'TrigEvtBrokerCmd',
 'VariableMatrix',
 'Expression',
 'ExtractVector',
 'Module state items',
 'Item attribute items',
 'Module events items',
 'Function item']

# Adding and accessing items

New items are added to an module using the `add_item` method. Modules are accessed from a hive using subscripting (array/dictionary access). Similarly, items are accessed from a module using subscripting.

In [14]:
func_item_type = test_module.get_item_type('Function Item')

var1 = test_module.add_item('Variable', 'var1')
var2 = test_module.add_item('variable', 'var2')
sig1 = test_module.add_item('Signal', 'sig1')
func = test_module.add_item(func_item_type, 'func1')

assert test_hive['TestWorker']['var1'].item_id == var1.ItEm_Id


## Getting items


In [22]:
assert test_hive['TestWorker']['var1'].item_id.lower() == 'testworker.var1'


item1, item2 = test_hive.get_items('testworker.var1', 'testworker2.var1')
assert item1.module.name=='testworker' and item2.module.name=='testworker2'

## Adding attributes

In [15]:
var1.add_attr("Text1", "Some text here")
var2.Text1 = "Some other text"

assert var2.Text1=="Some other text"

## Setting External items

In [16]:
import time

func.Expression = "ex1+ex2"
func.external_items = [var1, var2]
var1.Value = 3.0
var2.Value = 5.5

time.sleep(1.0)     #Let APIS propagate the values
assert func['Value'] == 8.5

## Setting and getting several items at once
It is possible to set several items at once, using the hive.set_values and hive.get_values methods. This is probably the fastest way to set many values at once. In the example below we use the `ItemVQT.from_dict` method to construct a sequence of items that are passed to the `.set_values` method.

In [17]:
from pyprediktoredgeclient.util import ItemVQT
from datetime import datetime, timedelta

vals = {'testworker.var1':2.0, 'testworker.var2':12.0}
vqt = ItemVQT.from_dict(vals, time='2022-03-24 01:00')       #make a list of ItemVQT
test_hive.set_values(vqt)

time.sleep(1.0)     #Let APIS propagate the values

assert func.value == 14
assert func.time == datetime(2022, 3, 24, 1, 0,0)

It is also possible to read several values at once from different modules using the `.get_values` method on the hive. This method returns a sequence of `ItemVQT` objects

In [18]:
readvals = test_hive.get_values(['testworker.var1', 'testworker.var2'])

dict_vals = dict((i.item_id, i.value) for i in readvals)  #transform the list of ItemVQT to a dict

assert dict_vals['testworker.var1'] == 2.0 and dict_vals['testworker.var2'] == 12.0

## Datetime attributes

Item attributes that have a datetime type (i.e. item.time) can be set either using a Python datetime.datetime or using an ISO8601 formatted string.

In [19]:
test2_var1 = test_module2.add_item("variable", "var1", value=1.0, time="2022-03-22 06:00")
assert test2_var1.time == datetime(2022, 3, 22, 6, 0)

test2_var1.time = datetime(2022, 3, 22, 8, 0)
assert test2_var1.time == datetime(2022, 3, 22, 8, 0)

test2_var1.time = "2022-03-22 10:00"
assert test2_var1.time == datetime(2022, 3, 22, 10, 0)


## Adding a logger module

In [20]:
if 'testloggerhive' in test_hive:
    logger_hive = test_hive.get_module('testloggerhive')
else:
    logger_hive = test_hive.add_module('ApisLogger', 'testloggerhive')

logger_hive.Historylenght_unit = "Year(s)"
logger_hive.Historylenght_x=10

## Set the new 'testlogger_hive' attribute on the items we added above

We set the 'testlogger_hive' attribute to true, starting the logging of the testworker2.var_1 and testworker_2.var2 items. After this we write a couple of different values to the items to record them in the Honeystore database

In [36]:
test2_var2 = test_module2.add_item("variable", "var2", value=1.0, time="2022-03-22 06:00")

test2_var1.add_attr("testloggerhive", True)
test2_var2.testloggerhive = True


t = datetime(2022, 3, 23)
for i in range(10):
    new_values = ItemVQT.from_dict({'testworker2.var1':i+2.0,'testworker2.var2':i*2.0, }, time=t)
    test_hive.set_values(new_values)
    t += timedelta(hours=2)
    time.sleep(1)

NameError: name 'test_module2' is not defined

# Reading raw history values from items

In [9]:
ts = test2_var1.read_raw(start=datetime(2022, 3, 23))

values = [vqt.value for vqt in ts if vqt.quality.isgood]
assert values == [2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0]

# Honeystore database management

The following section is focused on the management of Honeystore databases through the Python client library. The Honeystore management functions are quite similar to the Apis Hive management. The possibilities include:
* Connecting to Honeystore servers, both local and remote
* Listing and selecting databases
* Adding and removing databases
* Setting database properties and run-modes
* Listing and selecting database items.
* Adding and removing database items.
* Clearing item history
* Setting item attributes

Note that reading and writing time-series data is *not* covered in this section, but rather in the "Timeseries" part.

## Importing the package

The Honeystore management functions are located in the `honeystore` subpackage.

In [24]:
from pyprediktoredgeclient import honeystore
from pyprediktoredgeclient.util import RunningMode

## Connecting to a Honestore server.

The connection to the Honeystore server is done by instantinating the `Honeystore(host_name)` class. The default connection is to the local machine.

In [25]:
hs = honeystore.Honeystore()

## Listing the databases in the Honeystore server

The databases in the server can be listed with the `list_databases()` method or accessed with the `databases` property.

In [27]:
db_names = [db.name for db in hs.databases]
#assert 'testdb' not in db_names
assert 'testloggerhive' in db_names


## Accessing existing databases

In [30]:
testdb_logger = hs['testloggerhive']

In [35]:
hdb_it, = testdb_logger.items
hdb_it.name

'testworker2.var1'

## Creating a new database

New Databases are added with the `add_database` method

```
add_database(name, path, max_items=1000, cache_size=10040) method of pyprediktoredgeclient.honeystore.Honeystore instance
    Add a new database to the Honestore server.
    
    Arguments:
    name: str.       The name of the new database. 
    path: str.       The storage path for the database data
    max_items: int.  The inital maximum number of items in the database
    cache_size: int. The cache size in the database.
```

In the example below we add a new database, `test` to the server. The files are stored in c:\temp\testdb

In [24]:
testdb = hs.add_database('testdb', r"c:\temp\testdb")

## Accessing database properties

In [29]:
import os.path

datadir = testdb.DataDirPath
display(f"datadir='{datadir}'")
# assert os.path.samefile(datadir, os.path.join(r"c:\temp\testdb", r'test.dat'))

assert testdb.UsedItems == 0

NameError: name 'testdb' is not defined

## Adding new items to an database

New items are added using the `add_item` method on the database.


```
add_item(item, var_type=VariantType.R8, record_type=RecordType.Eventbased, resolution=1000, array_size=0, history_length=31536000) method of pyprediktoredgeclient.honeystore.Database instance
    Add a new item to the honeystore
    
    Arguments:
    item_name: a string or an object with an 'item_id' property
    var_type: The variable type to use. Either a string or a VariantType enum value
    record_type: The recordtype to use. Either a string or a RecordType enum value
    array_size: The arraylength of for vector or mulitidim data. If you pass a tuple here the elements
                    in the tuple describes the dimensions
    history_length: integer. Lenght of horizon in seconds
```

In [27]:
# Arrays and matrixes are not supported at the moment.          
# testdb.add_item('array', array_size=20)
# testdb.add_item('matrix', array_size=(5,5))
scalar = testdb.add_item('scalar')

assert len(testdb)==1

0


## Fetching items from the database



In [28]:
scalar_twin = testdb['scalar']

assert scalar.item_id == scalar_twin.item_id

## Setting the running mode of the database

There are four running modes available: 
* Online: The database is online in normal operation. Reading and writing can be done.
* Admin: The database is in administrative mode. No r/w, properties can be changed.
* Disabled: The database has been disabled. 
* OnlineNoCache: The database is on-line without caching operation. Reading and imports can be done, no write	

In [29]:
import time
display(testdb.mode)
assert testdb.mode.name=='Online'

testdb.mode = RunningMode.OnlineNoCache
time.sleep(0.5)
display(testdb.mode)
assert testdb.mode.name=='OnlineNoCache'

testdb.mode = "Online"
time.sleep(0.5)
display(testdb.mode)



<RunningMode.Online: 1>

<RunningMode.OnlineNoCache: 6>

<RunningMode.Online: 1>

## Deleting items from the database

In [30]:
del testdb['scalar']

assert len(testdb)==0

## Deleting a database

In [32]:
del hs['testdb']
del hs["testloggerhive"]


# Semantic services

The semantic services offers functions to import nodesets, list namespaces and save nodeset-files

In [20]:
from pyprediktoredgeclient import hive
test_hive = hive.Hive('Prediktor.ApisLoader.pytestinstance')
sem = test_hive.semantics_service

In [24]:
with open(".\Models\Opc.Ua.Di.NodeSet2.xml") as f1:
    mod1 = sem.load_namespace(f1, exchangerate=1000)

In [21]:
# with open(".\Models\windtypes.xml") as wt_file:
#     mod1 = sem.load_namespace(wt_file, exchangerate=1000, text1='kult')

with open(".\Models\instance.xml") as inst_file:
    mod1 = sem.load_namespace(inst_file, exchangerate=1000, nameoptions='Displayname', nametermination='Objects folder')


In [31]:
mod1.set_properties({'Name termination': 'Objects folder'})

In [34]:
r=range(1000)
isinstance(r,range)

True

## Deleting a module

Modules are deleted with the `del` keyword. The argument can either be a string or a module.

In [ ]:
del test_hive[test_module]

# hiveservices revisited - deleting a hive

When deleting a hive we remove the service from the system, deleting all modules and asscocited information.

Before we delete the hive we ensure that the `AutoDeleteDB` property is set to False on the logger module we as we want to keep the database in Honeystore for the next section

In [ ]:
pytestinstance.running = False
time.sleep(1.0)
hiveservices.remove_instance(pytestinstance)
